In [36]:
/* OLD ALGORITHM */
/*
// Propagate the water down one 'tick' at a time
val newActiveWater = ListBuffer[(Int, Int)]()
activeWater.foreach(waterPos => {
if (checkIsFree(grid, waterPos, 'S')) {
    newActiveWater += ((waterPos._1, waterPos._2+1))
    insert(grid, waterPos, '|')
} else {
    insert(grid, waterPos, '~')
    if (checkUpstream(grid, waterPos)) {
        newActiveWater += ((waterPos._1, waterPos._2-1))
    }
    // Check either side
    if (checkIsFree(grid, waterPos, 'E')) {
        val eastPos = (waterPos._1+1, waterPos._2)
        insert(grid, eastPos, '|')
        newActiveWater += eastPos
    }
    if (checkIsFree(grid, waterPos, 'W')) {
        val westPos = (waterPos._1-1, waterPos._2)
        insert(grid, westPos, '|')
        newActiveWater += westPos
    }
}
})
activeWater = newActiveWater
printGrid(grid)
*/

In [36]:
/* DEBUGGING
activeWater.foreach(waterPos => {
    print(waterPos + ": ")
    print(grid(waterPos._2)(waterPos._1))
    print(" Below: " + grid(waterPos._2+1)(waterPos._1))
    print(" Below free? " + checkIsFree(grid, waterPos, 'S'))
    print(" Left scan: " + scan(grid, waterPos, 'W'))
    print(" Right scan: " + scan(grid, waterPos, 'E'))
    println
})
*/

In [37]:
import scala.io.Source
import scala.collection.mutable.ListBuffer

import scala.io.Source
import scala.collection.mutable.ListBuffer

In [38]:
val filename = "input"
val lines = Source.fromFile(filename).getLines.toList

filename: String = "input"
lines: List[String] = List(
  "y=1595, x=454..478",
  "x=460, y=1350..1371",
  "y=891, x=577..590",
  "x=616, y=1387..1404",
  "y=545, x=517..532",
  "x=622, y=416..424",
  "x=600, y=903..907",
  "x=557, y=163..176",
  "y=926, x=667..674",
  "x=528, y=993..1009",
  "y=1504, x=608..618",
  "x=443, y=322..336",
  "x=630, y=749..762",
  "y=1620, x=471..478",
...

In [39]:
def parse(line: String) : Map[String, Range] = {
    line.split(", ")
        .map(_ split('='))
        .map(x => (x(0), x(1))).toMap
        .mapValues(v => v.split("\\.\\.").map(_ toInt))
        .mapValues(v => if (v.length == 1) Range(v.head, v.head+1) else Range(v.head, v.tail.head+1))
}

defined function parse

In [50]:
val coordRanges = lines.map(parse)
val minX = coordRanges.map(_("x").min).min-1
val maxX = coordRanges.map(_("x").max).max+1
val minY = 0
val minYClay = coordRanges.map(_("y").min).min
val maxY = coordRanges.map(_("y").max).max
val springPos = (500-minX, 0) //x=500, y=0

coordRanges: List[Map[String, Range]] = List(
  Map(
    "y" -> Range(1595),
    "x" -> Range(
      454,
      455,
      456,
      457,
      458,
      459,
      460,
      461,
      462,
      463,
      464,
...
minX: Int = 428
maxX: Int = 685
minY: Int = 0
minYClay: Int = 7
maxY: Int = 1984
springPos: (Int, Int) = (72, 0)

In [41]:
def printGrid(grid : Array[Array[Char]]) : Unit = {
    grid.foreach(line => println(line.mkString("")))
}

defined function printGrid

In [42]:
// This updates the grid in-place (PBR) to avoid redefining the grid many times
def insert(grid: Array[Array[Char]], coords: (Int, Int), character: Char) : Unit = {
    grid(coords._2)(coords._1) = character
}

defined function insert

In [43]:
// Checks if the square in the given direction is the given character
def checkCharacter(grid: Array[Array[Char]], coords: (Int, Int), direction: Char, character: Char) : Boolean = {
    direction match {
        case 'E' => grid(coords._2)(coords._1+1) == character
        case 'S' => grid(coords._2+1)(coords._1) == character
        case 'W' => grid(coords._2)(coords._1-1) == character
        case _ => false
    }
}

def checkIsFree(grid: Array[Array[Char]], coords: (Int, Int), direction: Char) : Boolean = {
    checkCharacter(grid, coords, direction, '.')
}

def checkIsClay(grid: Array[Array[Char]], coords: (Int, Int), direction: Char) : Boolean = {
    checkCharacter(grid, coords, direction, '#')
}

def checkIsRunningWater(grid: Array[Array[Char]], coords: (Int, Int), direction: Char) : Boolean = {
    checkCharacter(grid, coords, direction, '|')
}

def checkIsWater(grid: Array[Array[Char]], coords: (Int, Int), direction: Char) : Boolean = {
    Seq('|', '~').exists(character => checkCharacter(grid, coords, direction, character))
}

// Check a direction (left/right) and keep scanning until either clay '#' is hit or below is free
// Returns a tuple of the coordinates found and whether or not it has a free square below
def scan(grid: Array[Array[Char]], coords: (Int, Int), direction: Char) : ((Int, Int), Boolean) = {
    val directionModifier = direction match {
        case 'E' => 1
        case 'W' => -1
        case _ => 1
    }
    var (x, y) = coords
    while (!checkIsFree(grid, (x, y), 'S') && !checkIsRunningWater(grid, (x, y), 'S') && !checkIsClay(grid, (x, y), direction)) {
        x += directionModifier
    } 
    ((x,y), checkIsFree(grid, (x, y), 'S') || checkIsRunningWater(grid, (x, y), 'S'))
}

// TODO Don't think these are needed any more

// Checks if the square above was a source of running water ('|')
def checkUpstream(grid: Array[Array[Char]], coords: (Int, Int)) : Boolean = {
    grid(coords._2-1)(coords._1) == '|'
}

// Checks if the current square is at the floor of a bucket
def checkBucketFloor(grid: Array[Array[Char]], coords: (Int, Int)) : Boolean = {
    val (x, y) = coords
    val row = grid(y)
    val below = grid(y+1)
    val before = row.slice(0, x)
    val after = row.slice(x, grid.length)
    val bucketStart = before.lastIndexOf('#')
    val bucketEnd = coords._1 + after.indexOf('#')
    val floorBefore = below.slice(0, x)
    val floorAfter = below.slice(x, grid.length)
    val floorStart = Seq(before.lastIndexOf('.'), before.lastIndexOf('|'), before.lastIndexOf('~')).max
    val floorEnd = coords._1 + Seq(before.indexOf('.'), before.lastIndexOf('|'), before.lastIndexOf('~')).min
    // TODO Do this if it's needed later...
    true
}

// Checks if the entire "bucket" at this level is full
def checkBucketLevelFull(grid: Array[Array[Char]], coords: (Int, Int)) : Boolean = {
    val row = grid(coords._2)
    val before = row.slice(0, coords._1)
    val after = row.slice(coords._1, grid.length)
    val bucketStart = before.lastIndexOf('#')
    val bucketEnd = coords._1 + after.indexOf('#')
    val bucket = row.slice(bucketStart, bucketEnd)
    bucket.forall(character => character == '~')
}

defined function checkCharacter
defined function checkIsFree
defined function checkIsClay
defined function checkIsRunningWater
defined function checkIsWater
defined function scan
defined function checkUpstream
defined function checkBucketFloor
defined function checkBucketLevelFull

In [44]:
// Initialise the spring/clay in the grid
val grid = Array.fill(maxY+1, maxX-minX+1)('.')
insert(grid, springPos, '+')
coordRanges.foreach(coordMap => {
    for (y <- coordMap("y")) {
        for (x <- coordMap("x")) {
            insert(grid, (x-minX, y), '#')
        }
    }
})
//printGrid(grid)
var activeWater = scala.collection.mutable.Set((springPos._1, springPos._2+1))

grid: Array[Array[Char]] = Array(
  Array(
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
    '.',
...
activeWater: collection.mutable.Set[(Int, Int)] = Set((72, 1))

In [45]:
while (!activeWater.isEmpty) {
    println("Distance travelled: " + activeWater.map(x => x._2).min + "/" + maxY + "\r")
    val newActiveWater = scala.collection.mutable.Set[(Int, Int)]()
    activeWater.foreach(waterPos => {
        // Ensure we stop at the bottom of grid
        if (waterPos._2 == grid.length-1 || checkIsRunningWater(grid, waterPos, 'S')) {
            insert(grid, waterPos, '|')
        }
        // If the square below is free, move to that
        else if (checkIsFree(grid, waterPos, 'S')) {
            newActiveWater += ((waterPos._1, waterPos._2+1))
            insert(grid, waterPos, '|')
        } 
        // If the square below is not free, search sideways
        else if (!checkIsRunningWater(grid, waterPos, 'S')) {
            // For both sides, keep searching until clay is found or empty square below
            // TODO determine how to split this across ticks/setting active
            val squaresEitherSide = Seq('E', 'W').map(direction => scan(grid, waterPos, direction))
            var insertChar = '.'
            val xRange = {
                val xVals = squaresEitherSide.map(x => x._1._1)
                Range(xVals.min, xVals.max+1)
            }
            // If one or both of the edges have an empty square below, use this/these as the new active square
            if (squaresEitherSide.exists(x => x._2)) {
                insertChar = '|'
                newActiveWater ++= squaresEitherSide.filter(x => x._2)
                    .map({case ((x, y), _) => (x, y+1)})
            }
            // If this results in both sides being clay, make square above active
            else {
                insertChar = '~'
                newActiveWater += ((waterPos._1, waterPos._2-1))
            }
            // Update the characters for the scanned squares
            for (x <- xRange) {
                insert(grid, (x, waterPos._2), insertChar)
            }
        }
    })
    activeWater = newActiveWater
    //println(activeWater.mkString(", "))
    //printGrid(grid)
    //println
}

Distance travelled: 1/1984
Distance travelled: 2/1984
Distance travelled: 3/1984
Distance travelled: 4/1984
Distance travelled: 5/1984
Distance travelled: 6/1984
Distance travelled: 7/1984
Distance travelled: 8/1984
Distance travelled: 9/1984
Distance travelled: 10/1984
Distance travelled: 11/1984
Distance travelled: 12/1984
Distance travelled: 13/1984
Distance travelled: 14/1984
Distance travelled: 15/1984
Distance travelled: 16/1984
Distance travelled: 15/1984
Distance travelled: 14/1984
Distance travelled: 13/1984
Distance travelled: 12/1984
Distance travelled: 11/1984
Distance travelled: 10/1984
Distance travelled: 9/1984
Distance travelled: 8/1984
Distance travelled: 7/1984
Distance travelled: 8/1984
Distance travelled: 9/1984
Distance travelled: 10/1984
Distance travelled: 11/1984
Distance travelled: 12/1984
Distance travelled: 13/1984
Distance travelled: 14/1984
Distance travelled: 15/1984
Distance travelled: 16/1984
Distance travelled: 17/1984
Distance travelled: 18/1984
Distan

Distance travelled: 147/1984
Distance travelled: 148/1984
Distance travelled: 149/1984
Distance travelled: 150/1984
Distance travelled: 151/1984
Distance travelled: 152/1984
Distance travelled: 153/1984
Distance travelled: 154/1984
Distance travelled: 155/1984
Distance travelled: 156/1984
Distance travelled: 157/1984
Distance travelled: 158/1984
Distance travelled: 159/1984
Distance travelled: 160/1984
Distance travelled: 161/1984
Distance travelled: 162/1984
Distance travelled: 163/1984
Distance travelled: 164/1984
Distance travelled: 165/1984
Distance travelled: 166/1984
Distance travelled: 167/1984
Distance travelled: 168/1984
Distance travelled: 169/1984
Distance travelled: 170/1984
Distance travelled: 171/1984
Distance travelled: 170/1984
Distance travelled: 169/1984
Distance travelled: 168/1984
Distance travelled: 167/1984
Distance travelled: 166/1984
Distance travelled: 165/1984
Distance travelled: 164/1984
Distance travelled: 163/1984
Distance travelled: 162/1984
Distance trave

Distance travelled: 300/1984
Distance travelled: 301/1984
Distance travelled: 302/1984
Distance travelled: 303/1984
Distance travelled: 304/1984
Distance travelled: 305/1984
Distance travelled: 306/1984
Distance travelled: 307/1984
Distance travelled: 308/1984
Distance travelled: 309/1984
Distance travelled: 310/1984
Distance travelled: 311/1984
Distance travelled: 312/1984
Distance travelled: 313/1984
Distance travelled: 314/1984
Distance travelled: 313/1984
Distance travelled: 312/1984
Distance travelled: 311/1984
Distance travelled: 310/1984
Distance travelled: 309/1984
Distance travelled: 308/1984
Distance travelled: 307/1984
Distance travelled: 306/1984
Distance travelled: 305/1984
Distance travelled: 306/1984
Distance travelled: 307/1984
Distance travelled: 308/1984
Distance travelled: 309/1984
Distance travelled: 310/1984
Distance travelled: 311/1984
Distance travelled: 312/1984
Distance travelled: 313/1984
Distance travelled: 314/1984
Distance travelled: 315/1984
Distance trave

Distance travelled: 447/1984
Distance travelled: 448/1984
Distance travelled: 449/1984
Distance travelled: 450/1984
Distance travelled: 451/1984
Distance travelled: 452/1984
Distance travelled: 453/1984
Distance travelled: 454/1984
Distance travelled: 455/1984
Distance travelled: 456/1984
Distance travelled: 457/1984
Distance travelled: 458/1984
Distance travelled: 459/1984
Distance travelled: 460/1984
Distance travelled: 461/1984
Distance travelled: 460/1984
Distance travelled: 459/1984
Distance travelled: 458/1984
Distance travelled: 457/1984
Distance travelled: 456/1984
Distance travelled: 457/1984
Distance travelled: 458/1984
Distance travelled: 459/1984
Distance travelled: 460/1984
Distance travelled: 461/1984
Distance travelled: 462/1984
Distance travelled: 463/1984
Distance travelled: 464/1984
Distance travelled: 465/1984
Distance travelled: 464/1984
Distance travelled: 463/1984
Distance travelled: 462/1984
Distance travelled: 461/1984
Distance travelled: 460/1984
Distance trave

Distance travelled: 622/1984
Distance travelled: 623/1984
Distance travelled: 624/1984
Distance travelled: 625/1984
Distance travelled: 626/1984
Distance travelled: 627/1984
Distance travelled: 628/1984
Distance travelled: 629/1984
Distance travelled: 630/1984
Distance travelled: 631/1984
Distance travelled: 632/1984
Distance travelled: 633/1984
Distance travelled: 634/1984
Distance travelled: 635/1984
Distance travelled: 636/1984
Distance travelled: 637/1984
Distance travelled: 638/1984
Distance travelled: 639/1984
Distance travelled: 640/1984
Distance travelled: 641/1984
Distance travelled: 642/1984
Distance travelled: 643/1984
Distance travelled: 644/1984
Distance travelled: 645/1984
Distance travelled: 644/1984
Distance travelled: 643/1984
Distance travelled: 642/1984
Distance travelled: 641/1984
Distance travelled: 640/1984
Distance travelled: 639/1984
Distance travelled: 638/1984
Distance travelled: 637/1984
Distance travelled: 636/1984
Distance travelled: 635/1984
Distance trave

...

Distance travelled: 1263/1984
Distance travelled: 1264/1984
Distance travelled: 1265/1984
Distance travelled: 1266/1984
Distance travelled: 1297/1984
Distance travelled: 1298/1984
Distance travelled: 1299/1984
Distance travelled: 1298/1984
Distance travelled: 1297/1984
Distance travelled: 1296/1984
Distance travelled: 1295/1984
Distance travelled: 1294/1984
Distance travelled: 1293/1984
Distance travelled: 1292/1984
Distance travelled: 1291/1984
Distance travelled: 1290/1984
Distance travelled: 1289/1984
Distance travelled: 1288/1984
Distance travelled: 1289/1984
Distance travelled: 1290/1984
Distance travelled: 1291/1984
Distance travelled: 1292/1984
Distance travelled: 1293/1984
Distance travelled: 1294/1984
Distance travelled: 1295/1984
Distance travelled: 1296/1984
Distance travelled: 1297/1984
Distance travelled: 1298/1984
Distance travelled: 1299/1984
Distance travelled: 1300/1984
Distance travelled: 1301/1984
Distance travelled: 1302/1984
Distance travelled: 1303/1984
Distance t

Distance travelled: 1431/1984
Distance travelled: 1430/1984
Distance travelled: 1429/1984
Distance travelled: 1428/1984
Distance travelled: 1427/1984
Distance travelled: 1426/1984
Distance travelled: 1427/1984
Distance travelled: 1428/1984
Distance travelled: 1429/1984
Distance travelled: 1430/1984
Distance travelled: 1431/1984
Distance travelled: 1432/1984
Distance travelled: 1433/1984
Distance travelled: 1434/1984
Distance travelled: 1435/1984
Distance travelled: 1436/1984
Distance travelled: 1437/1984
Distance travelled: 1438/1984
Distance travelled: 1439/1984
Distance travelled: 1440/1984
Distance travelled: 1441/1984
Distance travelled: 1442/1984
Distance travelled: 1443/1984
Distance travelled: 1444/1984
Distance travelled: 1445/1984
Distance travelled: 1446/1984
Distance travelled: 1447/1984
Distance travelled: 1448/1984
Distance travelled: 1449/1984
Distance travelled: 1450/1984
Distance travelled: 1451/1984
Distance travelled: 1452/1984
Distance travelled: 1453/1984
Distance t

Distance travelled: 1577/1984
Distance travelled: 1578/1984
Distance travelled: 1579/1984
Distance travelled: 1580/1984
Distance travelled: 1581/1984
Distance travelled: 1582/1984
Distance travelled: 1583/1984
Distance travelled: 1582/1984
Distance travelled: 1581/1984
Distance travelled: 1580/1984
Distance travelled: 1579/1984
Distance travelled: 1578/1984
Distance travelled: 1577/1984
Distance travelled: 1576/1984
Distance travelled: 1575/1984
Distance travelled: 1574/1984
Distance travelled: 1573/1984
Distance travelled: 1572/1984
Distance travelled: 1571/1984
Distance travelled: 1572/1984
Distance travelled: 1573/1984
Distance travelled: 1574/1984
Distance travelled: 1575/1984
Distance travelled: 1576/1984
Distance travelled: 1577/1984
Distance travelled: 1578/1984
Distance travelled: 1579/1984
Distance travelled: 1580/1984
Distance travelled: 1581/1984
Distance travelled: 1582/1984
Distance travelled: 1583/1984
Distance travelled: 1584/1984
Distance travelled: 1585/1984
Distance t

Distance travelled: 1707/1984
Distance travelled: 1708/1984
Distance travelled: 1709/1984
Distance travelled: 1710/1984
Distance travelled: 1711/1984
Distance travelled: 1712/1984
Distance travelled: 1713/1984
Distance travelled: 1714/1984
Distance travelled: 1715/1984
Distance travelled: 1716/1984
Distance travelled: 1717/1984
Distance travelled: 1718/1984
Distance travelled: 1719/1984
Distance travelled: 1720/1984
Distance travelled: 1721/1984
Distance travelled: 1722/1984
Distance travelled: 1723/1984
Distance travelled: 1724/1984
Distance travelled: 1725/1984
Distance travelled: 1726/1984
Distance travelled: 1727/1984
Distance travelled: 1728/1984
Distance travelled: 1729/1984
Distance travelled: 1730/1984
Distance travelled: 1731/1984
Distance travelled: 1730/1984
Distance travelled: 1729/1984
Distance travelled: 1728/1984
Distance travelled: 1727/1984
Distance travelled: 1726/1984
Distance travelled: 1725/1984
Distance travelled: 1724/1984
Distance travelled: 1723/1984
Distance t

Distance travelled: 1897/1984
Distance travelled: 1896/1984
Distance travelled: 1895/1984
Distance travelled: 1896/1984
Distance travelled: 1897/1984
Distance travelled: 1898/1984
Distance travelled: 1897/1984
Distance travelled: 1896/1984
Distance travelled: 1895/1984
Distance travelled: 1894/1984
Distance travelled: 1893/1984
Distance travelled: 1892/1984
Distance travelled: 1891/1984
Distance travelled: 1890/1984
Distance travelled: 1891/1984
Distance travelled: 1892/1984
Distance travelled: 1893/1984
Distance travelled: 1894/1984
Distance travelled: 1895/1984
Distance travelled: 1896/1984
Distance travelled: 1897/1984
Distance travelled: 1898/1984
Distance travelled: 1899/1984
Distance travelled: 1900/1984
Distance travelled: 1901/1984
Distance travelled: 1902/1984
Distance travelled: 1903/1984
Distance travelled: 1904/1984
Distance travelled: 1905/1984
Distance travelled: 1906/1984
Distance travelled: 1907/1984
Distance travelled: 1908/1984
Distance travelled: 1909/1984
Distance t

In [53]:
//printGrid(grid)
// Get all of the water squares, minus those squares above the clay (top lines - 1 for spring)
grid.map(row => row.filter(x => x == '~' || x == '|').length).sum - minYClay + 1
grid.map(row => row.filter(x => x == '~').length).sum

res51_0: Int = 29741
res51_1: Int = 24198

In [48]:
// Write out the grid to a file
import java.io._
val file = new File("grid")
val writer = new BufferedWriter(new FileWriter(file))
try {
    grid.foreach(row => writer.write(row.mkString("").replace(".", " ") + "\n"))
}
finally {
  writer.close()
}

import java.io._
file: File = grid
writer: BufferedWriter = java.io.BufferedWriter@4446985e